In [1]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.num_layers = len(layers)
        self.weights = [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]
        self.biases = [np.random.randn(y, 1) for y in layers[1:]]

    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward_propagation(self, X):
        activations = [X]
        zs = []
        a = X

        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            zs.append(z)
            a = self.sigmoid(z)
            activations.append(a)

        return activations, zs

    def back_propagation(self, X, y, activations, zs):
        m = y.shape[1]
        gradients_w = [np.zeros(w.shape) for w in self.weights]
        gradients_b = [np.zeros(b.shape) for b in self.biases]

        delta = activations[-1] - y
        gradients_w[-1] = np.dot(delta, activations[-2].T) / m
        gradients_b[-1] = np.sum(delta, axis=1, keepdims=True) / m

        for l in range(2, self.num_layers):
            delta = np.dot(self.weights[-l + 1].T, delta) * self.sigmoid_derivative(zs[-l])
            gradients_w[-l] = np.dot(delta, activations[-l - 1].T) / m
            gradients_b[-l] = np.sum(delta, axis=1, keepdims=True) / m

        return gradients_w, gradients_b

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            activations, zs = self.forward_propagation(X)
            gradients_w, gradients_b = self.back_propagation(X, y, activations, zs)

            self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_w)]
            self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_b)]

            if epoch % 100 == 0:
                cost = np.mean(np.square(activations[-1] - y))
                print(f"Epoch {epoch}: Cost = {cost}")

    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return activations[-1]

# Example usage
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
y = np.array([[0, 1, 1, 0]])

nn = NeuralNetwork([2, 2, 1])  # 2 input neurons, 2 hidden neurons, 1 output neuron
nn.train(X, y, epochs=1000, learning_rate=0.1)

predictions = nn.predict(X)
print("Predictions:", predictions)


Epoch 0: Cost = 0.2640995223089507
Epoch 100: Cost = 0.2472296983425239
Epoch 200: Cost = 0.24646677546101747
Epoch 300: Cost = 0.2455820947655612
Epoch 400: Cost = 0.24454391303113096
Epoch 500: Cost = 0.24331601049922286
Epoch 600: Cost = 0.2418556116289475
Epoch 700: Cost = 0.24011285780473257
Epoch 800: Cost = 0.23803205285323134
Epoch 900: Cost = 0.23555605567382049
Predictions: [[0.42374991 0.52416073 0.53139822 0.55223759]]
